# 1. Token Seperation

In [1]:
import re

keywords = ["if", "and", "for", "while", "public", "void", "int", "static"]

operators = ["+", "-", "*", "/", "%"]

token_patterns = [
    (r"[ \t\n]+", None),
    (r"\d+", "INTEGER"),
    (r"[a-zA-Z_][a-zA-Z_0-9]*", "IDENTIFIER"),
    (r"[()-{},*;/=+%]", "PUNCTUATION"),
]


def tokenize(code):
    tokens = []
    while code:
        match = None
        for pattern, tag in token_patterns:
            regex = re.compile(pattern)
            match = regex.match(code)
            if match:
                text = match.group(0)

                if tag == "IDENTIFIER":
                    tag = "KEYWORD" if text in keywords else tag

                if tag == "PUNCTUATION":
                    tag = "OPERATOR" if text in operators else tag

                if tag:
                    tokens.append((tag, text))
                break

        if not match: #remember this is outside the for loop
            print("Error: Invalid Usage")
            break


        code = code[match.end():]
    return tokens

java_code="""
public static void main(){
    int a, b;
    int c = a - b;
}
"""

tokens = tokenize(java_code)
for t in tokens:
    print(f"< {t[0]}, {t[1]}>")


< KEYWORD, public>
< KEYWORD, static>
< KEYWORD, void>
< IDENTIFIER, main>
< PUNCTUATION, (>
< PUNCTUATION, )>
< PUNCTUATION, {>
< KEYWORD, int>
< IDENTIFIER, a>
< PUNCTUATION, ,>
< IDENTIFIER, b>
< PUNCTUATION, ;>
< KEYWORD, int>
< IDENTIFIER, c>
< PUNCTUATION, =>
< IDENTIFIER, a>
< OPERATOR, ->
< IDENTIFIER, b>
< PUNCTUATION, ;>
< PUNCTUATION, }>


----

# 2. Symbol tabel

In [3]:
def generate_symbol_tabel(code):
    symbol_tabel = {}
    lines = code.split(";")

    for line in lines:
        tokens = line.split()
        if len(tokens)>0:
            datatype = tokens[0]

            if '(' in line and ')' in line:
                function_return_type = tokens[0]
                function_name = tokens[1].split("(")[0]
                parameters = line.split("(")[1].split(")")[0].split(",")
                no_of_prarameters = len(parameters)
                parameters_dtypes = [i.split()[0] for i in parameters]
                symbol_tabel[function_name] = {"Retuen Type" : function_return_type,
                                               "Number of Parameters" : no_of_prarameters,
                                               "Parameters Datatypes" : parameters_dtypes}
            else:
                variables = "".join(tokens[1:]).split(",")
                for var in variables:
                    variable_name, *variable_val = var.split("=")
                    variable_val = variable_val[0] if variable_val else 0
                    symbol_tabel[variable_name] = {"Data type" : datatype,
                                                   "Value" : variable_val}

    for key, val in symbol_tabel.items():
        print(f"{key} ==> {val}")


input_code = """int a, b, c = 8;
                float d, e = 5.7;
                char f = "Hi";
                double Add(double x, double y);
"""
generate_symbol_tabel(input_code)


a ==> {'Data type': 'int', 'Value': 0}
b ==> {'Data type': 'int', 'Value': 0}
c ==> {'Data type': 'int', 'Value': '8'}
d ==> {'Data type': 'float', 'Value': 0}
e ==> {'Data type': 'float', 'Value': '5.7'}
f ==> {'Data type': 'char', 'Value': '"Hi"'}
Add ==> {'Retuen Type': 'double', 'Number of Parameters': 2, 'Parameters Datatypes': ['double', 'double']}


----

# 3. MacroProcessor 

In [17]:
def get_define_dict(input_code):
    define_dict = {}
    lines = input_code.split("\n")
    for line in lines:
        words = line.split()
        if words[0] == "#define":
            if '(' in words[-1] and ')' in words[-1]:
                fun_name = words[1].split('(')[0]
                params = words[1].split('(')[1].split(")")[0]
                params_list = params.split(',')
                operation = words[-1]
                define_dict[fun_name] = {'params_list' : params_list, 'operation': operation}
            else:
                identi = words[1]
                value = words[-1]
                define_dict[identi] = {'value': value}
    return define_dict

def generate_output(input_code, define_dict):
    output = ""
    line = input_code.split("\n")
    lines = [i.strip() for i in line]
    for line in lines:
        words = line.split()
        if words[0] == "#define":
            continue
        else:
            for j in words:
                for key in define_dict:
                    if key in j and '(' in j and ')' in j:
                        params = j.split('(')[1].split(')')[0]
                        params_list = params.split(',')
                        modified_operation = define_dict[key]['operation']
                        for idx, name in enumerate(define_dict[key]['params_list']):
                            modified_operation = modified_operation.replace(name, params_list[idx])
                        j = j.replace(f"{key}({params})", modified_operation)
                        string = j
                    elif key in j:
                        j = j.replace(key, define_dict[key][value])
                        string = j
                    else:
                        string = j
                output += string+' '
            output += '\n'


    return output

input_code ="""#include <stdio.h>
    #define PI 3.14
    #define ADD(x,y) (x+y)
    #define ADD3(x,y,z) (x+y+z)
    #define SQUARE(x) (x*x)
    void main(){
        int a, b, c, r, r_sq;
        c = ADD(a,b);
        r = ADD3(a,b,c);
        r_sq = SQUARE(r);
    }"""
define_dict = get_define_dict(input_code)
output = generate_output(input_code, define_dict)
print(output)


#include <stdio.h> 
void main(){ 
int a, b, c, r, r_sq; 
c = (a+b); 
r = (a+b+c); 
r_sq = (r*r); 
} 



-----------

# 5. DFA

In [20]:
number_of_states = int(input("Enter number of states: "))
dfa = {}

for i in range(number_of_states):
    state = input("Enter the state: ")
    a_trans = input("for a: ")
    b_trans = input("for b: ")
    dfa[state] = {"a": a_trans, "b": b_trans}
    print("-----------------------------------")
start_state = input("Enter start state: ")
final_state = input("Enter final state(s): ").split()
string = input("Enter the string: ")

curr_state = start_state

for char in string:
    if char in dfa[curr_state]:
        curr_state = dfa[curr_state][char]
    else:
        print("Striung is not accepted by DFA")
        break

if curr_state in final_state:
    print("Yes, Striung is accepted by DFA")
else:
    print("Striung is not accepted by DFA")


Enter number of states:  3
Enter the state:  e0
for a:  e0
for b:  e1


-----------------------------------


Enter the state:  e1
for a:  e0
for b:  e2


-----------------------------------


Enter the state:  e2
for a:  e0
for b:  e2


-----------------------------------


Enter start state:  e0
Enter final state(s):  e0 e1
Enter the string:  abbbab


Yes, Striung is accepted by DFA


---

# 7.Predictive Parsing

In [36]:
parsing_table = {
    "E": {
        "id": ["T", "E'"],
        "+": None,
        "*": None,
        "(": ["T", "E'"],
        ")": None,
        "$": None
    },
    "E'": {
        "id": None,
        "+": ["+", "T", "E'"],
        "*": None,
        "(": None,
        ")": [],
        "$": []
    },
    "T": {
        "id": ["F", "T'"],
        "+": None,
        "*": None,
        "(": ["F", "T'"],
        ")": None,
        "$": None
    },
    "T'": {
        "id": None,
        "+": [],
        "*": ["*", "F", "T'"],
        "(": None,
        ")": [],
        "$": []
    },
    "F": {
        "id": ["id"],
        "+": None,
        "*": None,
        "(": ["(", "E", ")"],
        ")": None,
        "$": None
    }
}

non_terminals = ['id', '*', '+', '(', ')', '$']
input_expr = ['id', '+', 'id', '*', 'id', '$']
stack = ["$", "E"]
print(f"{'STACK':<10} {'INPUT':<10} {'OUTPUT':<10}")
print(f"{''.join(stack):<10} {''.join(input_expr):<10} {'':<10}")

while stack != ['$']:
    last = stack.pop()
    stack += parsing_table[last][input_expr[0]][::-1]
    print(f"{''.join(stack):<10} {''.join(input_expr):<10} {last} -> {''.join(parsing_table[last][input_expr[0]]) if parsing_table[last][input_expr[0]] != [] else 'e' :<10}")
    while stack[-1] == input_expr[0]:
        if len(stack) == 1 and len(input_expr) == 1:
            print("Accepted")
            break
        stack.pop()
        input_expr.pop(0)
        print(f"{''.join(stack):<10} {''.join(input_expr):<10} {'':<10}")


STACK      INPUT      OUTPUT    
$E         id+id*id$            
$E'T       id+id*id$  E -> TE'       
$E'T'F     id+id*id$  T -> FT'       
$E'T'id    id+id*id$  F -> id        
$E'T'      +id*id$              
$E'        +id*id$    T' -> e         
$E'T+      +id*id$    E' -> +TE'      
$E'T       id*id$               
$E'T'F     id*id$     T -> FT'       
$E'T'id    id*id$     F -> id        
$E'T'      *id$                 
$E'T'F*    *id$       T' -> *FT'      
$E'T'F     id$                  
$E'T'id    id$        F -> id        
$E'T'      $                    
$E'        $          T' -> e         
$          $          E' -> e         
Accepted


----------

# 8. Simulation of loader

In [3]:
a = open("LAB8_input.txt")
a = a.readlines()
prog_name = input("Enter program name: ")
print(f"Obj for: {prog_name}")

for line in a:
    opcode = line.split('^')
    if opcode[0] == 'E':
        break
    elif opcode[0] == 'T':
        address = int(opcode[1], 16)
        for elemt in opcode[3:]:
            for i in range(0, len(elemt), 2):
                if elemt[i:i+2].strip() == "":
                    continue
                print(f"{address:06X} {elemt[i:i+2]}")
                address +=1


Obj for: df
001000 00
001001 10
001002 03
001003 07
001004 10
001005 09
002000 11
002001 11
002002 11


----------

# 9. YACC

----

# 10. Three address code 

In [8]:
import re
operators = ['+', '-', '*', '/']
user_expression = input("Enter the Expression: ")
m = re.split("([+-/*])", user_expression)
y = []
j = 1

while len(m) > 1:
    for word in m:
        if word in operators:
            y.append(''.join(m[0:3]))
            m.pop(0)
            m.pop(0)
            m[0] = "t" + str(j)
            j += 1

for i in range(0, len(y)):
    print(f"t{i+1} = {y[i]}")


Enter the Expression:  (a+b) - c /d


t1 = (a+b) 
t2 = t1- c 
t3 = t2/d


----